In [ ]:
from datetime import datetime
from grovepi import *
import grovepi
import time, math
from threading import Thread

# Bewegungssensor und LED
def LED_Bewegung():
    # Bewegungssensor mit digital port D8 verbinden.
    pir_sensor = 8
    # LED mit digital port D3 verbinden.
    led = 3

    # Input und Output definieren: Bewegungssensor ('pir_sensor') ist Input, LED ist Output
    pinMode(pir_sensor, "INPUT")
    pinMode(led, "OUTPUT")
    # Damit keine fehlerhaften Werte entstehen, wird eine Sekunde gewartet.
    time.sleep(1)
# Bewegung detektieren (im Zielbereich)
    while True:
        try:
            # Wenn der Bewegungssensor ('pir_sensor') Bewegung detektiert, wird das gemeldet (print) und das LED wird eingeschaltet (1= an).
            if digitalRead(pir_sensor):
                print 'Motion Detected'
                print 'LED ON!'
                digitalWrite(led, 1)
                time.sleep(.5)
            # Wenn der Bewegungssensor keine Bewegung detektiert, bleibt das LED ausgeschaltet (0= aus).
            else:
                digitalWrite(led, 0)
                print("LED OFF!")
                print '-'

            # Zwischen den Messungen muss immer kurz gewartet werden, damit der Ablauf funktioniert.
            time.sleep(1)

        # "Error" wird ausgegeben, wenn es einen Kommunikationsfehler gibt
        except IOError:
            print ("Error")
        # Wenn das Programm mit der Tastatur unterbrochen wird, muss das LED noch ausgeschaltet werden (0= aus), und die Schleife unterbrochen werden (break).
        except KeyboardInterrupt:
            digitalWrite(led, 0)
            break

# Geraeuschsensor
def Geraeuschsensor():
    print("Geraeuschsensor misst")

    # Geraeuschsensor mit analog port A0 verbinden.
    loudness_sensor = 0

    while True:
        try:
            # Lautstaerke messen
            # in eine globale Variable abspeichern
            global Lautstaerke
            Lautstaerke = grovepi.analogRead(loudness_sensor)

            # Die Lautstaerke wird im Ausgabefenster angezeigt.
            print("sensor_value = %d" % Lautstaerke)
            time.sleep(.25)

            # Wenn die Lautstaerke ueber einem vordefinierten Wert ist, wird die Variable Lautstaerke True, sonst False.
            # Solange es zu leise ist (False), wird weiter gemessen. Ist es genug laut (True), wird die Schleife abgebrochen (es wird nicht mehr gemessen).
            if Lautstaerke > 100:
                Lautstaerke = True
                break
            elif Lautstaerke < 100:
                Lautstaerke = False
            print(Lautstaerke)

        except IOError:
            print ("Error")

# Initialisierung des Geraueschsensors (ist nur einmal, beim Neustart, noetig. Grund ist der Ablauf des Programms).
def Initialise():
    Geraeuschsensor()

# Welche Uhrzeit? Globale Variablen Stunde und Minute.
# Uhrzeit des Raspberry Pis gilt (Achtung, das Raspberry Pi braucht dafuer eine Internetverbindung)
def Uhrzeit():
    global Stunde, Minute
    now = datetime.now()
    Stunde = int(now.hour)
    Minute = int(now.minute)

# Welches Datum? Globale Variablen Tag, Monat, Jahr.
# Datum des Raspberry Pis gilt (Achtung, das Raspberry Pi braucht dafuer eine Internetverbindung)
def Datum():
    global Tag, Monat, Jahr
    now = datetime.now()
    Tag = int(now.strftime("%d"))
    Monat = int(now.strftime("%m"))
    Jahr = int(now.year)

# Welche Temperatur? Globale Variable aktuelle Temperatur.
def Temperaturabfrage():
    print("Temperatursensor misst")

    # Temperatursensor mit digital port D4 verbinden.
    sensor = 4

    # Unser Temperatursensor hat zwei Typen, einen blauen und einen weissen Sensor. Wir brauchen den weissen Sensor.
    blue = 0
    white = 1

    # Der erste Parameter ist der Port, an dem der Temperatursensor angeschlossen ist (sensor), der zweite Parameter ist der Typ des Sensors (hier white).
    [temp, humidity] = grovepi.dht(sensor, white)
    # Das Modul math braucht man fuer Mathematische Funktionen
    # isnan(x) gibt den Wert True zurueck, wenn x ein NaN (Not a Number) ist. Wir moechten aber eine Zahl, also False.
    if math.isnan(temp) == False and math.isnan(humidity) == False:
        # Die aktuelle Temperatur wird in einer globalen Variable zwischengespeichert, damit der Wert auch ausserhalb der Funktion verwendet werden kann.
        # Die Variable aktuelle Temperatur wird so formatiert, dass sie auf dem Display angezeigt werden kann. Zahlen hinter dem Komma werden nicht beachtet (int).
        global aktTemp
        aktTemp = int(float("%.02f" % (temp)))

# Display anschalten.
# Nur einmal noetig, naemlich wenn man das ganze Programm neu startet.
def Displayan():
    global grovepi, display

    # Display mit digital port D5 verbinden.
    display = 5
    # Display als Output definieren.
    grovepi.pinMode(display, "OUTPUT")

    # Display initialisieren
    grovepi.fourDigit_init(display)
    time.sleep(.5)

    # Auf das niedrigste Helligkeitslevel setzen (0).
    grovepi.fourDigit_brightness(display, 0)
    time.sleep(.5)

# Display ausschalten
def Displayaus():
    grovepi.fourDigit_off(display)

# Darstellung der Uhrzeit auf dem Display
def Display_Uhrzeit():
    # Uhrzeit wird abgefragt (Funktion oben definiert)
    Uhrzeit()
    print("Display zeigt Uhrzeit: ")
    print(Stunde, Minute)
    # Anzeige auf Display
    grovepi.fourDigit_score(display, Stunde, Minute)
    time.sleep(3)
    Displayaus()
    time.sleep(1)

# Darstellung des Datums auf dem Display
def Display_Datum():
    # Datum wird abgefragt (Funktion oben definiert)
    Datum()
    print("Display zeigt Datum: ")
    print(Tag, Monat)
    print(Jahr)
    # leading_zero= 0 bedeutet, dass im Display in fehlenden Feldern (beim Datum 1.05. fehlt z.B. ein Null --> 01.05) eine Null angezeigt wird.
    leading_zero = 0
    # Anzeige auf Display (zuerst Tag und Monat, dann Jahr)
    grovepi.fourDigit_score(display, Tag, Monat)
    time.sleep(3)
    grovepi.fourDigit_number(display, Jahr, leading_zero)
    time.sleep(3)
    Displayaus()
    time.sleep(1)

# Darstellung der Temperatur auf dem Display
def Display_Temperatur():
    # Temperatur wird abgefragt (Funktion oben definiert)
    Temperaturabfrage()
    print("Display zeigt Temperatur: ")
    print(aktTemp)
    # leading_zero= 1 bedeutet, dass im Display in fehlenden Feldern (z.B. 24, da fehlen zwei Felder 0024) nichts angezeigt wird.
    leading_zero = 1
    # Anzeige auf Display
    grovepi.fourDigit_number(display, aktTemp, leading_zero)
    time.sleep(3)
    Displayaus()
    time.sleep(1)

# Darstellung Text ("Cool HSLU") auf dem Display
def Display_Text():
    # Der erste Parameter ist das Display (auf was soll es angezeigt werden), dann die Position (Index 0 ist z.B. die erste Zahl), dann der Code fuer den Buchstaben (im Internet zu finden).
    grovepi.fourDigit_segment(display, 0, 57)  # 57 = C
    grovepi.fourDigit_segment(display, 1, 63)  # 63 = O
    grovepi.fourDigit_segment(display, 2, 63)  # 63 = O
    grovepi.fourDigit_segment(display, 3, 56)  # 56 = L
    time.sleep(3)
    grovepi.fourDigit_segment(display, 0, 118)  # 118 = H
    grovepi.fourDigit_segment(display, 1, 0x6d)  # 0x6d= S
    grovepi.fourDigit_segment(display, 2, 56)  # 56 = L
    grovepi.fourDigit_segment(display, 3, 0x3e)  # 0x3e= U
    time.sleep(3)
    Displayaus()

# Nachdem etwas auf dem Display angezeigt wurde, wird ein Neustart durchgefuehrt. Dabei wird die Startzeit zurueckgesetzt und der Geraeuschsensor neu gestartet.
def Neustart():
    global start_time
    start_time = time.time()
    Geraeuschsensor()

# Der Programmablauf, der alle Funktionen miteinander verbindet:
def Ablauf():
    start="Ja"
    # Die Startzeit wird mit dem Time-Modul gemessen und in einer globalen Variablen abgelegt.
    global start_time
    start_time=time.time()
    # Die globale Variable counter sorgt dafuer, dass die Angaben nacheinander angezeigt werden, ausser man wartet zulange, dann startet es von vorne.
    global counter
    counter=0
    while start=="Ja":
        try:
            # Wenn man schnell genug ist (nicht laenger als 4 Sekunden zwischen den Angaben wartet), kann man nacheinander alle Werte abfragen.
            # Wenn man zwischen den Werten laenger als 4 Sekunden wartet, beginnt der Ablauf von vorne (bei der Uhrzeit).
            # Die while-Schleife hat zwei Moeglichkeiten:
            #       Wartezeit kleiner als 4 Sekunden und ein lautes Geraeusch
            #       ODER
            #       Wartezeit kleiner als 4 Sekunden und eine Bewegung detektiert
            # Ist eine dieser Bedingungen erfuellt, wird etwas auf dem Display angezeigt (je nach Counter, dieser legt die Reihenfolge fest).
            # Nach der Displayanzeige wird die Startzeit neu festgelegt und der Counter aktualisiert.
            # Der Geraeuschsensor beginnt wieder zu messen, die while-Schleife wird vorgesetzt (continue).
            while (time.time()-start_time<4) and Lautstaerke==True or (time.time()-start_time<4 and digitalRead(pir_sensor)):
                time.sleep(1)
                if counter==0:
                    Display_Uhrzeit()
                    counter=1
                    Neustart()
                    continue
                if counter==1:
                    Display_Datum()
                    counter=2
                    Neustart()
                    continue
                if counter==2:
                    Display_Temperatur()
                    counter=3
                    Neustart()
                    continue
                if counter==3:
                    Display_Text()
                    counter=0
                    # Hier gab es immer fehlerhafte Werte, weshalb der Geraeuschsensor zweimal gestartet wird, mit einer Pause dazwischen.
                    Neustart()
                    time.sleep(1)
                    Neustart()
                    continue
            # Falls nicht rechtzeitig geklatscht wird und/oder gar nicht geklatscht/bewegt wird:
            # Counter wird auf 0 zurueckgesetzt, damit der Ablauf beim naechsten Klatschen wieder von vorne bei der Uhrzeit beginnt.
            # Die Startzeit wird zurueckgesetzt.
            # Die while-Schleife wird nach einer kurzen Wartezeit fortgesetzt (continue)
            counter = 0
            start_time = time.time()
            time.sleep(.5)

        # Im Falle eines KeyboardInterrupts wird das Display ausgeschaltet.
        except KeyboardInterrupt:
            grovepi.fourDigit_off(display)

# Das Display wird initialisiert.
Displayan()
# Im Fall, dass man das Programm mit der Tastatur stoppt (KeyboardInterrupt), wird das Display ausgeschaltet.
if KeyboardInterrupt:
    grovepi.fourDigit_off(display)

# Der Geraeuschsensor wird initialisiert.
Initialise()

# Mit Threading werden zwei Funktionen gleichzeitig ausgefuehrt: LED_Bewegung und Ablauf. Das heisst, der Bewegungssensor und Geraueschsensor messen gleichzeitig.
# Der Bewegungssensor hat einen Einfluss auf das LED und das Display. Der Geraeuschsensor beeinflusst nur das Display.
if __name__=='__main__':
    Thread(target=LED_Bewegung).start()
    Thread(target=Ablauf).start()
